## Check GPU Access

In [1]:
!nvidia-smi

Fri Dec 20 11:50:43 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Connect with Google Drive

In [2]:
!pip install gdown

In [3]:
! rm -rf video.mp4


### Download the video

In [4]:
import gdown

# Update with your file's specific ID
file_id = "1NNhyO_PUrfy3dVat_BmlYaBSDCLIzo2Q"
url = f"https://drive.google.com/uc?id={file_id}"

output = "vehicle-counting.mp4"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1NNhyO_PUrfy3dVat_BmlYaBSDCLIzo2Q
To: /kaggle/working/vehicle-counting.mp4
100%|██████████| 35.3M/35.3M [00:02<00:00, 17.5MB/s]


'vehicle-counting.mp4'

In [5]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [6]:
SOURCE_VIDEO_PATH = "/kaggle/working/vehicle-counting.mp4"

## Install YOLOv8

In [7]:
# Pip install method (recommended)

!pip install "ultralytics<=8.3.40"

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6037.7/8062.4 GB disk)


In [8]:
# settings
MODEL = "yolov8x.pt"

### 

In [9]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:01<00:00, 77.3MB/s] 


YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


In [10]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source={SOURCE_VIDEO_PATH}

/kaggle/working
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs

video 1/1 (frame 1/538) /kaggle/working/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 54.7ms
video 1/1 (frame 2/538) /kaggle/working/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 27.3ms
video 1/1 (frame 3/538) /kaggle/working/vehicle-counting.mp4: 384x640 4 cars, 2 trucks, 27.2ms
video 1/1 (frame 4/538) /kaggle/working/vehicle-counting.mp4: 384x640 5 cars, 1 truck, 27.2ms
video 1/1 (frame 5/538) /kaggle/working/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 27.2ms
video 1/1 (frame 6/538) /kaggle/working/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 27.2ms
video 1/1 (frame 7/538) /kaggle/working/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 27.2ms
video 1/1 (frame 8/538) /kaggle/working/vehicle-counting.mp4: 384x640 5 cars, 1 truck, 27.2ms
video 1/1 (frame 9/538) /kaggle/working/vehicle-co

### Save Json Output and Frames

In [ ]:
import os
import json
from ultralytics import YOLO
import cv2

OUTPUT_DIR = os.path.join(HOME, "runs/detect/predict")
FRAME_OUTPUT_DIR = os.path.join(HOME, "frames")
JSON_OUTPUT_DIR = os.path.join(HOME, "frame_data")

# Create directories for frames and JSON output
os.makedirs(FRAME_OUTPUT_DIR, exist_ok=True)
os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

# Run YOLO detection
model = YOLO("yolov8x.pt")  # Load YOLO model
results = model.predict(source=SOURCE_VIDEO_PATH, conf=0.25, save=True)

# Process each frame and save detections
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_filename = os.path.join(FRAME_OUTPUT_DIR, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_filename, frame)  # Save frame

    # Get detections for the current frame
    detections = results.pandas().xyxy[0].to_dict(orient="records")

    # Save detections to a JSON file
    json_filename = os.path.join(JSON_OUTPUT_DIR, f"frame_{frame_count:04d}.json")
    with open(json_filename, "w") as json_file:
        json.dump(detections, json_file, indent=4)

    frame_count += 1

cap.release()
print(f"Frames and JSON data saved to {FRAME_OUTPUT_DIR} and {JSON_OUTPUT_DIR}")